# Features 3, 4, 5

In [3]:
import pandas as pd 

ratings = pd.read_csv('https://raw.githubusercontent.com/nchichilidze/RS-with-GE/main/preprocessed_movielens_1m/ratings.csv')
ratings = ratings.iloc[: , 1:]

movies =  pd.read_csv('https://raw.githubusercontent.com/nchichilidze/RS-with-GE/main/preprocessed_movielens_1m/movies.csv')

In [4]:
movies['ML_rating'] = 0
movies['ML_votes'] = 0

In [10]:
for index, row in movies.iterrows():
    if (index % 1000 == 0):
        print(index)
    
    movie_id = row['movie_id'] 
    
    ratings_data = ratings[ratings['movie_id'] == movie_id]
    ratings_sum = ratings_data['rating'].sum()
    ratings_count = len(ratings_data)
    
    if (ratings_count != 0):
        ml_rating = int(ratings_sum) / int(ratings_count)
    else: 
        ml_rating = 0
        
    movies.at[index, 'ML_rating'] = ml_rating
    movies.at[index, 'ML_votes'] = ratings_count

0
1000
2000
3000


In [14]:
movies['ML_rating'].value_counts()

3    2114
2    1003
4     420
0     177
1     159
5      10
Name: ML_rating, dtype: int64

In [15]:
movies[movies['ML_rating'] == 5]

,Unnamed: 0,movie_id,title,genres,year,ML_rating,ML_votes
777,777,787,"Gate of Heavenly Peace, The",documentary,1995,5,3
977,977,989,Schlafes Bruder (Brother of Sleep),drama,1995,5,1
1762,1762,1830,Follow the Bitch,comedy,1998,5,1
3103,3103,3172,Ulysses (Ulisse),adventure,1954,5,1
3164,3164,3233,Smashing Time,comedy,1967,5,2
3211,3211,3280,"Baby, The",horror,1973,5,1
3313,3313,3382,Song of Freedom,drama,1936,5,1
3538,3538,3607,One Little Indian,comedy drama western,1973,5,1
3587,3587,3656,Lured,crime,1947,5,1
3811,3811,3881,Bittersweet Motel,documentary,2000,5,1


In [16]:
genres_map = {'drama': 0,
              'comedy' : 1, 
              'action' : 2, 
              'thriller': 3,
              'romance': 4, 
              'horror': 5, 
              'adventure': 6,
              'sci-fi': 7,
              'children\'s': 8,
              'crime': 9,
              'war': 10,
              'documentary': 11, 
              'musical': 12,
              'mystery': 13,
              'animation': 14,
              'fantasy': 15,
              'western': 16, 
              'film_noir': 17,
              }

def encode_genres(genres): 
    encoded = '' 
    words = ''
    if (genres != None):
        words = genres.split()
    encoded = []
    for word in words: 
        index = genres_map.get(word)
        encoded.append(index)
    
    return encoded

In [18]:
movies['encoded'] = None

for index, row in movies.iterrows():
    genres_str = row['genres']
    genres_list = encode_genres(genres_str)
    movies.at[index, 'encoded'] = genres_list

In [20]:
users = list(ratings['user_id'].unique())
user_favourite_movies = pd.DataFrame(); 
user_favourite_movies['user'] = None
user_favourite_movies['favourite_movies'] = None

rating_threshold = 4 

for user_id in users: 
    movies_rated_by_user = ratings[ratings['user_id'] == user_id]
    highly_rated = movies_rated_by_user.loc[movies_rated_by_user['rating'] >= rating_threshold]['movie_id'].values
    
    new_row = {'user': user_id, 'favourite_movies': list(highly_rated)}
    user_favourite_movies = user_favourite_movies.append(new_row, ignore_index = True)

In [22]:
# generate a list of genres 

user_favourite_movies['genres'] = None 

for index, row in user_favourite_movies.iterrows(): 
    favourite_movies = list(row['favourite_movies'])
    favourite_genres = []
    for movie in favourite_movies: 
        movie_genres = list(movies[movies['movie_id'] == movie]['encoded'])
        if (movie_genres != None): 
            favourite_genres += movie_genres
    user_favourite_movies.at[index, 'genres'] = favourite_genres
        

In [23]:
user_favourite_movies.head()

,user,favourite_movies,genres
0,1,"[1193, 3408, 2355, 1287, 2804, 594, 919, 595, ...","[[0], [0], [14, 8, 1], [2, 6, 0], [1, 0], [14,..."
1,2,"[1357, 3068, 1537, 2194, 648, 2268, 3468, 1210...","[[0, 4], [0], [1], [2, 9, 0], [2, 6, 13], [9, ..."
2,3,"[3421, 1394, 104, 2735, 1210, 1079, 1615, 1291...","[[1], [1], [1], [2, 6, 1], [2, 6, 4, 7, 10], [..."
3,4,"[3468, 2951, 1214, 1036, 260, 2028, 480, 1198,...","[[0], [2, 16], [2, 5, 7, 3], [2, 3], [2, 6, 15..."
4,5,"[2987, 2333, 1175, 2337, 1535, 1392, 866, 2770...","[[6, 14, None], [0], [1, 7], [0], [0, 4], [1, ..."


In [24]:
import collections 

def get_favourites(genres): 
    all_genres = []
    for list_of_genres in genres: 
        all_genres += list_of_genres
    
    counter  = collections.Counter(all_genres)
    top_3_with_count = counter.most_common(3)
    top_3 = []
    for keyval in top_3_with_count: 
        key = keyval[0]
        top_3 += [key]
    return top_3


user_favourite_movies['favourite_genres'] = None
for index, row in user_favourite_movies.iterrows(): 
    genres = row['genres']
    favourites = get_favourites(genres)
    user_favourite_movies.at[index, 'favourite_genres'] = favourites    
    

In [25]:
user_favourite_movies

,user,favourite_movies,genres,favourite_genres
0,1,"[1193, 3408, 2355, 1287, 2804, 594, 919, 595, ...","[[0], [0], [14, 8, 1], [2, 6, 0], [1, 0], [14,...","[0, 8, 14]"
1,2,"[1357, 3068, 1537, 2194, 648, 2268, 3468, 1210...","[[0, 4], [0], [1], [2, 9, 0], [2, 6, 13], [9, ...","[0, 2, 3]"
2,3,"[3421, 1394, 104, 2735, 1210, 1079, 1615, 1291...","[[1], [1], [1], [2, 6, 1], [2, 6, 4, 7, 10], [...","[6, 1, 2]"
3,4,"[3468, 2951, 1214, 1036, 260, 2028, 480, 1198,...","[[0], [2, 16], [2, 5, 7, 3], [2, 3], [2, 6, 15...","[2, 7, 0]"
4,5,"[2987, 2333, 1175, 2337, 1535, 1392, 866, 2770...","[[6, 14, None], [0], [1, 7], [0], [0, 4], [1, ...","[0, 1, 3]"
...,...,...,...,...
6035,6036,"[589, 3006, 3008, 2064, 2069, 590, 593, 594, 5...","[[2, 7, 3], [0], [3], [1, 11], [0], [6, 0, 16]...","[0, 1, 3]"
6036,6037,"[3006, 2064, 593, 903, 904, 908, 909, 910, 912...","[[0], [1, 11], [0, 3], [13, 3], [13, 3], [0, 3...","[0, 3, 1]"
6037,6038,"[1419, 3088, 232, 1136, 1148, 1183, 2146, 3548...","[[0], [1], [1, 0], [1], [14, 1], [0, 4, 10], [...","[1, 0, 4]"
6038,6039,"[588, 2067, 1416, 3022, 2080, 3035, 3037, 901,...","[[14, 8, 1, 12], [0, 4, 10], [0, 12], [1], [14...","[1, 12, 0]"


In [26]:
def has_intersection(list1, list2): 
    list1 = list1[0]
    list2 = list2[0]
    for item in list1: 
        if item in list2: 
            return True
    return False

In [28]:
data = ratings.copy(deep='True')

In [30]:
data.head()

,user_id,movie_id,rating,is_favourite_genre
0,1,1193,5,0
1,1,661,3,0
2,1,914,3,0
3,1,3408,4,0
4,1,2355,5,0


In [31]:
# check if each movie is of the user's favourite genre 

data['is_favourite_genre'] = 0
for index, row in data.iterrows(): 
    if (index % 100000 == 0): 
        print(index)
    user = row['user_id']
    movie = row['movie_id']
    user_favourite_genres = user_favourite_movies[user_favourite_movies['user'] == user]['favourite_genres']
    movie_genres = movies[movies['movie_id'] == movie]['encoded']
    
    if (has_intersection(movie_genres.values, user_favourite_genres.values)): 
        data.at[index, 'is_favourite_genre'] = 1

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000


In [33]:
data['is_favourite_genre'].value_counts()

1    822445
0    177764
Name: is_favourite_genre, dtype: int64

In [34]:
data['ML_rating'] = 0
data['ML_votes'] = 0
    
for index, row in data.iterrows():
    if (index % 100000 == 0): 
        print(index)
    movie_id = row['movie_id']
    
    movie_data = movies[movies['movie_id'] == movie_id]
    ML_rating = movie_data['ML_rating']
    ML_votes = movie_data['ML_votes']
    
    data.at[index, 'ML_rating'] = ML_rating
    data.at[index, 'ML_votes'] = ML_votes
    

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000


In [35]:
data.head()

,user_id,movie_id,rating,is_favourite_genre,ML_rating,ML_votes
0,1,1193,5,1,4,1725
1,1,661,3,1,3,525
2,1,914,3,0,4,636
3,1,3408,4,1,3,1315
4,1,2355,5,1,3,1703


In [37]:
data = data.drop(columns=['user_id', 'movie_id', 'rating'])
data.to_csv('ratings_3_5.csv', index=False)